In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load Dataset
file_path = "/kaggle/input/twitter-us-airline/Twitter_US_Airline/Tweets.csv"
df = pd.read_csv(file_path)

# Preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip()

df["cleaned_text"] = df["text"].apply(preprocess_text)

# Encode labels
sentiment_mapping = {"negative": 0, "neutral": 1, "positive": 2}
df["label"] = df["airline_sentiment"].map(sentiment_mapping)

# Split data
X_train, X_test, y_train, y_test = train_test_split(df["cleaned_text"], df["label"], test_size=0.2, random_state=42)

# Tokenize text for Word2Vec
sentences = [text.split() for text in df["cleaned_text"]]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
vocab_size = len(word2vec_model.wv)
embedding_dim = 100

# Tokenize text for LSTM/GRU input
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Prepare embedding matrix from Word2Vec
embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))
for word, index in tokenizer.word_index.items():
    if index < vocab_size and word in word2vec_model.wv:
        embedding_matrix[index] = word2vec_model.wv[word]

# Convert labels to categorical
num_classes = len(sentiment_mapping)
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# Build LSTM/GRU Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size + 1, 
                    output_dim=embedding_dim, 
                    weights=[embedding_matrix], 
                    input_length=max_length, 
                    trainable=False))

# Choose either LSTM or GRU by uncommenting one of the following:
# LSTM Layer
# model.add(LSTM(units=128, return_sequences=False))
# GRU Layer
model.add(GRU(units=128, return_sequences=False))

model.add(Dropout(0.5))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Model Summary
model.summary()

# Train the model
epochs = 10
batch_size = 32
history = model.fit(X_train_pad, y_train_cat, 
                    validation_data=(X_test_pad, y_test_cat), 
                    epochs=epochs, 
                    batch_size=batch_size)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test_cat)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predictions and Classification Report
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test_cat, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred_classes, target_names=sentiment_mapping.keys()))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │         640,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 640,400 (2.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 640,400 (2.44 MB)

Epoch 1/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.6222 - loss: 0.9555 - val_accuracy: 0.6452 - val_loss: 0.9015
Epoch 2/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6242 - loss: 0.9230 - val_accuracy: 0.6452 - val_loss: 0.8987
Epoch 3/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6185 - loss: 0.9292 - val_accuracy: 0.6452 - val_loss: 0.8941
Epoch 4/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6209 - loss: 0.9261 - val_accuracy: 0.6452 - val_loss: 0.8944
Epoch 5/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6217 - loss: 0.9251 - val_accuracy: 0.6452 - val_loss: 0.8979
Epoch 6/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6243 - loss: 0.9225 - val_accuracy: 0.6452 - val_loss: 0.8957
Epoch 7/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6277 - loss: 0.9173 - val_accuracy: 0.6452 - val_loss: 0.8972
Epoch 8/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6213 - loss: 0.9258 - val_accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
